In [ ]:
%config IPCompleter.greedy=True

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
import import_ipynb
from functions import get_detailed_aggregate_data
from functions import get_raw_data
from functions import get_aggregate_functions
from visuals import colors
from visuals import map_position_to_color

In [ ]:
import os
import requests
import pandas

# Get parameters
# SEASON = os.environ["FPL_SEASON"]
SEASON = '2018-19'

# Get current gameweek
# URL = "https://fantasy.premierleague.com/api/bootstrap-static"
# DATA = requests.get(URL).json()
# CURR_GW = DATA['next-event'] - 1
CURR_GW = 38

BASE_PATH = 'C:/Users/aelek/source/antoniaelek/fantasy-premier-league/'
BASE_PATH = '../'

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go


def get_agg_features(features, aggregates):
    agg_features = []
    for feature in features:
        for agg in aggregates:
            agg_features.append(agg + '_' + feature)
    return agg_features


def get_trace(df, x_metrics, y_metrics, color):
    return go.Bar(
        x = df[x_metrics],        
        y = df[y_metrics],
        text = df['name'],        
        #mode = 'markers',        
        marker=dict(color=color),
        hovertemplate = "<b>%{text}</b><br><br>" +
            y_metrics+": %{y:.2f}</br>"+
#             x_metrics+": %{x}</br>"+
            "<extra></extra>")


def get_data(df, x_metrics, y_metrics, color):
    data = []
    for x in x_metrics:
        for y in y_metrics:
            data.append(get_trace(df, x, y, color))
    
    for el in data[1:]:
        el['visible'] = 'legendonly'
    
    return data


def get_layout(df, x_metrics, y_metrics, title):  
    buttons=[]
    i = 0
    for x in x_metrics:
        for y in y_metrics:
            template = [False] * len(y_metrics)
            template[i] = True
            buttons.append(dict(label = y, method = 'update', args = [{'visible': template}]))        
            i+=1
        
    updatemenus = list([
        dict(active=0,
             pad = {'r': 0, 't': 10},
             x = 0,
             y = 1.18,
             xanchor = 'left',
             buttons=buttons)])

    layout = go.Layout(
        hovermode = 'closest',
        showlegend=False,
        updatemenus=updatemenus, 
        title=go.layout.Title(
            text=title,
                font=dict(
                    size=21
                )
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text='',
                font=dict(
                    size=18
                )
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text='',
                font=dict(
                    size=18
                )
            )
        )
    )
    return layout


def get_figure(df, x_metric, y_metrics, title = '', color='rgba(101,255,71, 0.4)'):    
    data = get_data(df, [x_metric], y_metrics, color)
    layout = get_layout(df, [x_metric], y_metrics, title)
    
    return go.Figure(data=data, layout=layout)

In [ ]:
def filter_achievements(points_dict):
    return {k:v for (k,v) in points_dict.items() if  v > 0}


def filter_errors(points_dict):
    return {k:v for (k,v) in points_dict.items() if  v < 0}


def get_errors(df, points_dict):
    errors = pandas.Series()
    for e in filter_errors(points_dict):
        tmp = (df[e]*(0-points_dict[e]))
        errors = errors.add(tmp, fill_value=0)
    return errors


def get_achievements(df, points_dict):
    achievements = pandas.Series()
    for e in filter_achievements(points_dict):
        tmp = (df[e]*(points_dict[e]))
        achievements = achievements.add(tmp, fill_value=0)
    return achievements

In [ ]:
agg_data = get_detailed_aggregate_data(BASE_PATH, SEASON)
raw_data = get_raw_data(BASE_PATH, SEASON)
df = pandas.merge(agg_data, raw_data, on='name', how='outer')
df['minutes_points'] = df['minutes']/45
df.sort_values('web_name', inplace=True)

In [ ]:
goalkeepers = df[df.position == 'Goalkeeper'].copy(deep=True)
defenders = df[df.position == 'Defender'].copy(deep=True)
midfielders = df[df.position == 'Midfielder'].copy(deep=True)
forwards = df[df.position == 'Forward'].copy(deep=True)

In [ ]:
points_metrics = ['goals_scored','assists','own_goals','clean_sheetes','goals_conceded',
                  'penalties_missed','penalties_saved','minutes','yellow_cards']

defensive_metrics = ['clean_sheets','saves','penalties_saved','recoveries','clearances_blocks_interceptions','tackles',
                     'goals_conceded','own_goals','penalties_conceded','errors_leading_to_goal','errors_leading_to_goal_attempt']

creativity_metrics=['assists','big_chances_created','big_chances_missed',
                    'attempted_passes','completed_passes','key_passes','dribbles','open_play_crosses']

attack_metrics=['goals_scored','winning_goals','penalties_missed','target_missed','tackled','offside']

general_metrics=['minutes','red_cards','yellow_cards','fouls',
               'bonus','bps','total_points','ea_index','ict_index','influence','creativity','threat']

other_metrics=['cost','selected','loaned_in','loaned_out','transfers_in','transfers_out','transfers_balance']

points = {'assists':3,
          'own_goals':-2,
          'penalties_missed':-2,
          'penalties_saved':5,
          'minutes':1/45,
          'yellow_cards':-1,
          'red_cards':-3}

In [ ]:
game_metrics_gkp = points_metrics + general_metrics + defensive_metrics
game_metrics_gkp = set(game_metrics_gkp)

points_gkp = points.copy()
points_gkp['goals_scored']=6
points_gkp['saves']=0.5
points_gkp['clean_sheets']=4
points_gkp['goals_conceded']=-1

goalkeepers['achievements']=get_achievements(goalkeepers,points_gkp)
goalkeepers['errors']=get_errors(goalkeepers,points_gkp)

goalkeepers=goalkeepers[goalkeepers['achievements']>0]

In [ ]:
game_metrics_def = points_metrics + general_metrics + defensive_metrics
game_metrics_def = set(game_metrics_def)

points_def = points.copy()
points_def['goals_scored']=6
points_def['clean_sheets']=4
points_def['goals_conceded']=-1

defenders['achievements']=get_achievements(defenders,points_def)
defenders['errors']=get_errors(defenders,points_def)

defenders=defenders[defenders['achievements']>0]

In [ ]:
game_metrics_mid = points_metrics + general_metrics + creativity_metrics + attack_metrics
game_metrics_mid = set(game_metrics_mid)

points_mid = points.copy()
points_mid['goals_scored']=5
points_mid['clean_sheets']=1
points_mid['goals_conceded']=0

midfielders['achievements']=get_achievements(midfielders,points_mid)
midfielders['errors']=get_errors(midfielders,points_mid)

midfielders=midfielders[midfielders['achievements']>0]

# Goalkeepers

In [ ]:
y1 = ['achievements']
for a in filter_achievements(points_gkp).keys(): y1.append(a)

gk1 = get_figure(goalkeepers, 'web_name', y1, 'achievements', map_position_to_color('Goalkeeper'))
plotly.offline.iplot(gk1)

In [ ]:
y2 = ['errors']
for e in filter_errors(points_gkp).keys(): y2.append(e)
gk2 = get_figure(goalkeepers[(goalkeepers['saves']>1)], 'web_name', y2, 'errors', map_position_to_color('Goalkeeper'))
plotly.offline.iplot(gk2)

In [ ]:
goalkeepers['value'] = goalkeepers['achievements']-goalkeepers['errors']
gkp3 = get_figure(goalkeepers, 'web_name', ['value'], 'value', map_position_to_color('Goalkeeper'))
plotly.offline.iplot(gkp3)

# Defenders

In [ ]:
y1 = ['achievements']
for a in filter_achievements(points_def).keys(): y1.append(a)

def1 = get_figure(defenders, 'web_name', y1, 'achievements', map_position_to_color('Defender'))
plotly.offline.iplot(def1)

In [ ]:
y2 = ['errors']
for e in filter_errors(points_def).keys(): y2.append(e)
def2 = get_figure(defenders, 'web_name', y2, 'errors', map_position_to_color('Defender'))
plotly.offline.iplot(def2)

In [ ]:
defenders['value'] = defenders['achievements']-defenders['errors']
def3 = get_figure(defenders, 'web_name', ['value'], 'value', map_position_to_color('Defender'))
plotly.offline.iplot(def3)

# Midfielders

In [ ]:
y1 = ['achievements']
for a in filter_achievements(points_def).keys(): y1.append(a)
mid1 = get_figure(midfielders, 'web_name', y1, 'achievements', map_position_to_color('Midfielder'))
plotly.offline.iplot(mid1)

In [ ]:
y2 = ['errors']
for e in filter_errors(points_mid).keys(): y2.append(e)
mid2 = get_figure(midfielders, 'web_name', y2, 'errors', map_position_to_color('Midfielder'))
plotly.offline.iplot(mid2)

In [ ]:
midfielders['value'] = midfielders['achievements']-midfielders['errors']
mid3 = get_figure(midfielders, 'web_name', ['value'], 'value', map_position_to_color('Midfielder'))
plotly.offline.iplot(mid3)